In [75]:
import sklearn
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn import metrics

from joblib import dump, load
from joblib import Parallel, delayed

In [39]:
#pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 16
INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [40]:
# Imports the Google Cloud client library
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_final_project_kshitijm"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_final_project_kshitijm connected.


In [1]:
import torch

In [2]:
# pip install transformers
from transformers import pipeline
# specific_model = pipeline(model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

2023-05-20 23:21:09.131778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 23:21:10.231191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-20 23:21:10.231314: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [55]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [32]:
prompt = ["Chatgpt has made my life really good","Openai stock stock is going down"]
token=tokenizer(prompt, padding='max_length', max_length=256, truncation=True, return_tensors='pt')

In [34]:
output=model(**token)

In [35]:
output.logits

tensor([[-2.4181, -2.8373,  5.7378],
        [ 4.3566, -1.3070, -3.4122]], grad_fn=<AddmmBackward0>)

In [37]:
import torch.nn.functional as F
probs=F.softmax(output.logits)
probs=probs.detach().numpy()
print(probs)

[[2.8689814e-04 1.8867313e-04 9.9952447e-01]
 [9.9612242e-01 3.4564650e-03 4.2109768e-04]]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


### Predicting sentiments for our dataframe

In [41]:
df_filt_2020=pd.read_parquet('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2020_Topics.parquet')
df_filt_2020.head()

,url,date,language,title,text,cleaned_text,article_source,clean_title,title_tokens,cleaned_text_tokens,year_pub,month,month-year,flag_relevant,num_tokens,rake_phrases_articles,rake_phrases_joined,final_topic
0,https://fusionscienceacademy.com/artificial-in...,2020-01-30,en,Artificial Intelligence (AI) in Social Media ...,\n\nArtificial Intelligence (AI) in Social Med...,Artificial Intelligence AI in Social MediaMar...,2026 – Fusion Science Academy,Artificial Intelligence (AI) in Social Media ...,"['artificial', 'intelligence', 'ai', 'social',...","['artificial', 'intelligence', 'ai', 'social',...",2020,1,Jan 2020,1,22283,[burkert fluid control systems emerson electri...,burkert fluid control systems emerson electric...,6
1,https://health.economictimes.indiatimes.com/ne...,2020-01-10,en,artificial intelligence: Researchers develop A...,\n\nartificial intelligence: Researchers devel...,artificial intelligence: Researchers develop ...,None,artificial intelligence: Researchers develop A...,"['artificial', 'intelligence', 'researchers', ...","['artificial', 'intelligence', 'researchers', ...",2020,1,Jan 2020,1,8087,[economic times ethealthworldhome news hospita...,economic times ethealthworldhome news hospital...,1
2,https://heraldpublicist.com/bet-gil-on-ai-fina...,2020-01-15,en,Bet Gil on AI Final Fantasy Tactics Matches in...,\n\nBet Gil on AI Final Fantasy Tactics Matche...,Bet Gil on AI Final Fantasy Tactics Matches i...,Herald Publicist,Bet Gil on AI Final Fantasy Tactics Matches in...,"['bet', 'gil', 'ai', 'final', 'fantasy', 'tact...","['bet', 'gil', 'ai', 'final', 'fantasy', 'tact...",2020,1,Jan 2020,1,4458,[hilarious twitch streamnewstechnologycricketp...,hilarious twitch streamnewstechnologycricketpo...,1
3,https://honestversion.com/2020/01/24/growth-of...,2020-01-24,en,Growth of Cloud Telecommunication AI market in...,\n\nGrowth of Cloud Telecommunication AI marke...,Growth of Cloud AI market in global industry...,Honest Version,Growth of Cloud Telecommunication AI market in...,"['growth', 'cloud', 'ai', 'market', 'global', ...","['growth', 'cloud', 'ai', 'market', 'global', ...",2020,1,Jan 2020,1,6048,[cloud telecommunication ai market strategic a...,cloud telecommunication ai market strategic as...,6
4,https://marketresearchsheets.com/2020/01/31/gl...,2020-01-31,en,Global Artificial Intelligence as a Service Ma...,\n\nGlobal Artificial Intelligence as a Servic...,Global Artificial Intelligence as a Service M...,Market Research Sheets,Global Artificial Intelligence as a Service Ma...,"['global', 'artificial', 'intelligence', 'serv...","['global', 'artificial', 'intelligence', 'serv...",2020,1,Jan 2020,1,10098,[service market trendsyou may also like news g...,service market trendsyou may also like news gl...,6


In [58]:
df_filt_2020_samp=df_filt_2020.sample(1000)

In [59]:
df_filt_2020_samp

,url,date,language,title,text,cleaned_text,article_source,clean_title,title_tokens,cleaned_text_tokens,year_pub,month,month-year,flag_relevant,num_tokens,rake_phrases_articles,rake_phrases_joined,final_topic
5775,https://time.com/5792613/ai-art/?utm_source=fe...,2020-03-02,en,How Artists Are Using AI to Confront Modern An...,\n\nHow Artists Are Using AI to Confront Moder...,How Artists Are Using AI to Confront Modern ...,Time,How Artists Are Using AI to Confront Modern An...,"['artists', 'ai', 'confront', 'modern', 'anxie...","['artists', 'ai', 'confront', 'modern', 'story...",2020,3,Mar 2020,1,12940,[costa rica cote divoire cuba dominica dominic...,costa rica cote divoire cuba dominica dominica...,1
7786,https://nysenewstimes.com/global-artificial-in...,2020-03-03,en,Global Artificial Intelligence in Medicine Mar...,\n\nGlobal Artificial Intelligence in Medicine...,Global Artificial Intelligence in Medicine Ma...,"IBM, Esko Bionics, Cyrcadia Health, Atomwise –...",Global Artificial Intelligence in Medicine Mar...,"['global', 'artificial', 'intelligence', 'medi...","['global', 'artificial', 'intelligence', 'medi...",2020,3,Mar 2020,1,6618,[sfeb1 2345678 9101112131415 16171819202122 23...,sfeb1 2345678 9101112131415 16171819202122 232...,6
13239,https://redhillcouncillors.co.uk/main-news/502...,2020-05-03,en,Artificial Intelligence In Beauty Market 2020-...,\n\nArtificial Intelligence In Beauty Market 2...,Artificial Intelligence In Beauty Market 2020...,Redhill Market Analytics,Artificial Intelligence In Beauty Market 2020-...,"['artificial', 'intelligence', 'beauty', 'mark...","['artificial', 'intelligence', 'beauty', 'mark...",2020,5,May 2020,1,10381,[content redhill market analytics redhill rese...,content redhill market analytics redhill resea...,6
9629,https://amarketresearchgazette.com/global-arti...,2020-04-03,en,Global Artificial Intelligence Software System...,\n\nGlobal Artificial Intelligence Software Sy...,Global Artificial Intelligence Software Syste...,Market Research Gazette,Global Artificial Intelligence Software System...,"['global', 'artificial', 'intelligence', 'soft...","['global', 'artificial', 'intelligence', 'soft...",2020,4,Apr 2020,1,10163,[download best wordpress themes free downloadd...,download best wordpress themes free downloaddo...,6
28015,https://prnewsleader.com/coronavirus/1421402/g...,2020-11-01,en,Global Artificial Intelligence-Emotional Recog...,\n\nGlobal Artificial Intelligence-Emotional R...,Global Artificial Recognition Market Expecte...,PRnews Leader,Global Artificial Intelligence-Emotional Recog...,"['global', 'artificial', 'intelligence', 'emot...","['global', 'artificial', 'recognition', 'marke...",2020,11,Nov 2020,1,7391,[2026 archives november 2020 october 2020 sept...,2026 archives november 2020 october 2020 septe...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18283,http://www.business.itbusinessnet.com/2020/07/...,2020-07-07,en,BISTelligence 2020 Unveils Cavalcade of Manufa...,\n\nBISTelligence 2020 Unveils Cavalcade of Ma...,BISTelligence 2020 Unveils Cavalcade of Manuf...,| IT Business Net,BISTelligence 2020 Unveils Cavalcade of Manufa...,"['bistelligence', 'unveils', 'cavalcade', 'man...","['bistelligence', 'unveils', 'cavalcade', 'man...",2020,7,Jul 2020,1,11220,[categoriescategories select category 3d1 3d c...,categoriescategories select category 3d1 3d ca...,2
10466,https://jewishlifenews.com/uncategorized/artif...,2020-04-11,en,Artificial Intelligence (AI) As a Service Mark...,\nArtificial Intelligence (AI) As a Service Ma...,Artificial Intelligence AI As a Service Marke...,"Top Players Are Google, Amazon web services, I...",Artificial Intelligence (AI) As a Service Mark...,"['artificial', 'intelligence', 'ai', 'service'...","['artificial', 'intelligence', 'ai', 'service'...",2020,4,Apr 2020,1,8288,[2020jewish market reports jewish market resea...,2020jewish market reports jewish market resear...,6
450,https://www.wfmz.com/news/pr_newswir

In [ ]:
batch_size = 10
sentiment_probs=[]

for i in range(0, len(df_filt_2020_samp), batch_size):
    # Get the batch of articles
    batch_articles = list(df_filt_2020_samp[i:i+batch_size])
    print(batch_articles)
    
    # Tokenize the batch of articles
    inputs = tokenizer(batch_articles, padding='max_length', max_length=500 ,truncation=True, return_tensors='pt')
    
    # Forward pass through the model
    outputs = model(**inputs)
    
    # Access the embeddings or other output
    probs=F.softmax(output.logits)
    probs=probs.detach().numpy()
    sentiment_probs.append(probs)
    print(probs)

In [83]:
from tqdm import tqdm

def classify_sentiment(text):
    # Split or truncate the text if it exceeds the maximum sequence length
    # max_seq_length = 512
    token=tokenizer(text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
    output=model(**token)
    probs=F.softmax(output.logits)
    probs=probs.detach().numpy()
    return probs


In [84]:
# Apply the classification to the dataframe with progress tracker
tqdm.pandas()
df_filt_2020_samp=df_filt_2020_samp.sample(100)
df_filt_2020_samp['sentiment_text'] = df_filt_2020_samp['cleaned_text'].progress_apply(classify_sentiment)


  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 100/100 [00:15<00:00,  6.56it/s]


In [ ]:
pd.options.display.max_colwidth=None
df_filt_2020_samp[['cleaned_text','sentiment_text']].sample(5)

In [89]:
df_filt_2020_samp['sentiment_label']=df_filt_2020_samp['sentiment_text'].apply(lambda x: np.argmax(x[0]))
df_filt_2020_samp['sentiment_label'].value_counts()

1    71
2    24
0     5
Name: sentiment_label, dtype: int64

### Running on 2020 dataset

In [94]:
df_filt_2020['sent_probs']=df_filt_2020['cleaned_text'].progress_apply(classify_sentiment)
df_filt_2020['sent_label']=df_filt_2020['sent_probs'].progress_apply(lambda x: np.argmax(x[0]))

  0%|          | 0/32667 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 32667/32667 [00:00<00:00, 294287.02it/s]


In [96]:
df_filt_2020['sent_label'].value_counts()

1    22548
2     9429
0      690
Name: sent_label, dtype: int64

In [98]:
%%time
df_filt_2020.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2020_Topics_Sentiments.csv')                

CPU times: user 57.5 s, sys: 392 ms, total: 57.9 s
Wall time: 1min 22s


In [ ]:
# df_filt_2021=pd.read_parquet('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2021_Topics.parquet')
# df_filt_2022=pd.read_parquet('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2022_Topics.parquet')
# df_filt_2023=pd.read_parquet('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2023_Topics.parquet')